# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv("../output_data/city_data_df.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,288.88,59,0,3.60,US,1655103142
1,1,puebloviejo,10.9938,-74.2844,298.28,77,32,2.26,CO,1655103143
2,2,busselton,-33.6500,115.3333,289.30,69,85,7.05,AU,1655103145
3,3,san andres,12.5847,-81.7006,300.16,89,75,2.57,CO,1655103147
4,4,bytow,54.1706,17.4919,287.15,74,19,3.34,PL,1655103148
...,...,...,...,...,...,...,...,...,...,...
580,580,salalah,17.0151,54.0924,306.20,66,75,6.69,OM,1655103976
581,581,bundaberg,-24.8500,152.3500,291.05,64,0,2.80,AU,1655103977
582,582,aykhal,66.0000,111.5000,292.54,26,6,5.59,RU,1655103979
583,583,osoyoos,49.0331,-119.4524,288.57,94,100,2.36,CA,1655103980


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# create lists for heatmap
humidity = city_data_df["Humidity"]
city_loc = city_data_df[["Lat","Lng"]]

# create heatmap layer on gmap fig
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_loc, weights=humidity,
                                dissipating=False, max_intensity=100, 
                                point_radius = 2)
# add layer
fig.add_layer(heat_layer)

# check it exists
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# create df with desired weather conditions and drop null values
clean_city_df  = city_data_df.loc[(city_data_df["Max Temp"]>70)
                            &(city_data_df["Max Temp"]<80)
                            &(city_data_df["Wind Speed"]<10)
                            &city_data_df["Cloudiness"]==0,:].dropna()

clean_city_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,288.88,59,0,3.60,US,1655103142
1,1,puebloviejo,10.9938,-74.2844,298.28,77,32,2.26,CO,1655103143
2,2,busselton,-33.6500,115.3333,289.30,69,85,7.05,AU,1655103145
3,3,san andres,12.5847,-81.7006,300.16,89,75,2.57,CO,1655103147
4,4,bytow,54.1706,17.4919,287.15,74,19,3.34,PL,1655103148
...,...,...,...,...,...,...,...,...,...,...
580,580,salalah,17.0151,54.0924,306.20,66,75,6.69,OM,1655103976
581,581,bundaberg,-24.8500,152.3500,291.05,64,0,2.80,AU,1655103977
582,582,aykhal,66.0000,111.5000,292.54,26,6,5.59,RU,1655103979
583,583,osoyoos,49.0331,-119.4524,288.57,94,100,2.36,CA,1655103980


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df= clean_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,kodiak,US,57.7900,-152.4072,
1,puebloviejo,CO,10.9938,-74.2844,
2,busselton,AU,-33.6500,115.3333,
3,san andres,CO,12.5847,-81.7006,
4,bytow,PL,54.1706,17.4919,
...,...,...,...,...,...
580,salalah,OM,17.0151,54.0924,
581,bundaberg,AU,-24.8500,152.3500,
582,aykhal,RU,66.0000,111.5000,
583,osoyoos,CA,49.0331,-119.4524,


In [11]:
# create small df for api for loop testing and generation
hotel_head= hotel_df.head()


# create base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters for url search
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# Iterate through cities
for index, row in hotel_head.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    # try:
    hotel_head.loc[index, "Hotel Name"] = hotel_name[0]['name']
    print(f"Closest hotel is {hotel_name[0]['name']}.")
    hotel_head.loc[index, 'Hotel Name'] = hotel_name[0]['name']
    
    # except (KeyError, IndexError):
    # print("Missing field/result... skipping.")
    
    # print("------------")

# Display hotel names on dataframe
# hotel_head



Retrieving results for city: kodiak
Missing field/result... skipping.
------------
Retrieving results for city: puebloviejo
Missing field/result... skipping.
------------
Retrieving results for city: busselton
Missing field/result... skipping.
------------
Retrieving results for city: san andres
Missing field/result... skipping.
------------
Retrieving results for city: bytow
Missing field/result... skipping.
------------


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))